In [1]:
import tensorflow as tf
import numpy as np
import pprint as pp

tf.logging.set_verbosity(tf.logging.ERROR)
tf.set_random_seed(777)

In [3]:
input_data = list(set('hihello'))

h = [1, 0, 0, 0, 0]
e = [0, 1, 0, 0, 0]
i = [0, 0, 1, 0, 0]
l = [0, 0, 0, 1, 0]
o = [0, 0, 0, 0, 1]

# h:0, e:1, i:2, l:3, o:4 --> one-hot encoding

x_data = np.array([[h, i, h, e, l, l, o]], dtype=np.float32)
pp.pprint(x_data)

array([[[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]]], dtype=float32)


In [4]:
sequence_length = 7
hidden_size = 5  # one-hot으로 나와야 하기 때문에 5(h, e, i, l, o)
batch_size = 1  # 한줄만 입력하기 때문에 batch_size = 1

# rnn_cell은 GRUCell(rnn_size) 사용 가능
# cell = tf.contrib.rnn.GRUCell(num_units=hidden_size)
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)

outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval(session=sess))
print(outputs)

array([[[-0.00820011, -0.12516846, -0.00109155, -0.08578427,
         -0.02479585],
        [-0.09453642, -0.14250152, -0.02844694, -0.13483033,
          0.04651831],
        [-0.07265618, -0.2996451 , -0.01728353, -0.2039874 ,
         -0.01324248],
        [ 0.01648019, -0.17692621, -0.05124753, -0.1765648 ,
         -0.04826915],
        [ 0.00338528, -0.06204902, -0.09092198, -0.08830062,
         -0.02259476],
        [ 0.00506152,  0.08547659, -0.13553241,  0.01971849,
          0.02411455],
        [ 0.06973629,  0.05258247, -0.02265765, -0.1168239 ,
          0.10626922]]], dtype=float32)
Tensor("rnn/transpose_1:0", shape=(1, 7, 5), dtype=float32)


In [28]:
idx2char = ['h', 'i', 'e', 'l', 'o']
char_list = {}
for idx, char in enumerate(idx2char):
    char_list[char] = idx
char = 'hihell'
x_data = [[int(char_list[c]) for c in char]]
x_data

[[0, 1, 0, 2, 3, 3]]

In [29]:
y_char = 'ihello'
y_data = [[int(char_list[c]) for c in y_char]]
y_data

[[1, 0, 2, 3, 3, 4]]

In [52]:
tf.reset_default_graph()
sequence_length = len(y_data[0])
hidden_size = len(idx2char) # input dim
batch_size = 1

x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

X = tf.placeholder(tf.float32, shape=[1, sequence_length, hidden_size]) # X_one_hot
Y = tf.placeholder(tf.int32, shape=[None, sequence_length])  # Y label

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)
print(X_one_hot)
print(X)

Tensor("one_hot:0", shape=(1, 6, 5), dtype=float32)
Tensor("Placeholder:0", shape=(1, 6, 5), dtype=float32)


In [53]:
# cost/loss function
weights = tf.ones([batch_size, sequence_length], dtype=tf.float32)

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)  # 각 sequence의 loss를 평균한 값
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

In [60]:
prediction = tf.argmax(outputs, axis=2) # one-hot을 label 값으로 변환

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# training the model
for i in range(2000):
    cost_val, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
    result = sess.run(prediction, feed_dict={X: x_one_hot})
    
    if i % 20 == 0:
        print(i, "loss:", "%.9f" % cost_val, "prediction:", result, "true Y:", y_data)
    
        # print char using idc
        result_str = [idx2char[c] for c in np.squeeze(result)] # result를 1차원으로 축소하여 문자열 리스트 만들기
        print("\tPrediction str:", "".join(result_str)) # 예측한 라벨을 문자열로 출력

0 loss: 1.597315073 prediction: [[2 3 3 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ellllo
20 loss: 0.807377219 prediction: [[1 0 2 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihelll
40 loss: 0.701444507 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
60 loss: 0.666903913 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
80 loss: 0.656582236 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
100 loss: 0.652033687 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
120 loss: 0.649236679 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
140 loss: 0.647223413 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
160 loss: 0.645661414 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
180 loss: 0.644396424 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]


1620 loss: 0.630833566 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
1640 loss: 0.630788267 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
1660 loss: 0.630743742 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
1680 loss: 0.630699992 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
1700 loss: 0.630657077 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
1720 loss: 0.630615056 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
1740 loss: 0.630573630 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
1760 loss: 0.630533099 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
1780 loss: 0.630493164 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
	Prediction str: ihello
1800 loss: 0.630453944 prediction: [[1 0 2 3 3 4]] true Y: [[1, 

'hihell'를 입력했더니 예측값으로 ihello를 도출하였고, 코스트 값은 0.63이다.